In [53]:
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage
from IPython.display import SVG
import ipywidgets as widgets
from ipywidgets import interact
from pathlib import Path
import json
from cgr.filepaths import filepaths
from itertools import chain

In [ ]:
def draw_reaction(rxn: str, sub_img_size: tuple = (300, 200), use_smiles: bool = True):    
    rxn = Chem.rdChemReactions.ReactionFromSmarts(rxn, useSmiles=use_smiles)
    return Draw.ReactionToImage(rxn, useSVG=True, subImgSize=sub_img_size)

def draw_molecule(smiles: str, size: tuple = (200, 200), hilite_atoms : tuple = tuple()) -> str:
    '''
    Draw molecule to svg string

    Args
    ----
    mol:str
        Molecule SMILES
    stoich:int
        Stoichiometric coefficient
    size:tuple
        (width, height)
    hilite_atoms:tuple
        Atom indices to highlight
    auto_scl:bool
        If True, scales molecule image width proportional
        to log(# of atoms)
    '''
    mol = Chem.MolFromSmiles(smiles)
    
    # Catch failed MolFromSmiles
    if mol is None: 
        mol = Chem.MolFromSmiles(smiles, sanitize=False)

    drawer = Draw.MolDraw2DSVG(*size)
    drawer.DrawMolecule(mol, highlightAtoms=hilite_atoms)
    
    drawer.FinishDrawing()
    img = drawer.GetDrawingText()

    return img

def show_highlighted_grid(smiles: str, rcs: list[list], legends: list[str], n_cols: int, sub_img_size: tuple = (200, 200)):
    mgrid = MolsToGridImage(
        mols=[Chem.MolFromSmiles(smi) for smi in smiles],
        highlightAtomLists=rcs,
        molsPerRow=n_cols,
        legends=legends,
        subImgSize=sub_img_size
    )
    display(mgrid)

In [10]:
krs = Path(filepaths.data) / "raw" / "sprhea_240310_v3_mapped_no_subunits.json"
with open(krs, 'r') as f:
    krs = json.load(f)

In [11]:
decarb = {k: v for k,v  in krs.items() if v['min_rule'] == 'rule0024'}

In [49]:
kopts = list(decarb.keys())

@interact
def show_decarbs(key = widgets.Dropdown(value=kopts[0], options=kopts)):    
    rxn = decarb[key]
    print(rxn['imt_rules'])
    print(rxn['rhea_ids'])
    smiles = rxn['smarts'].split('>>')[0]
    rc = rxn['reaction_center'][0]
    display(SVG(draw_molecule(smiles, hilite_atoms=rc, size=(500, 300))))

interactive(children=(Dropdown(description='key', options=('14', '55', '121', '734', '1067', '1162', '1565', '…

In [ ]:
# stuff = [
#     (k, rxn['smarts'].split('>>')[0], rxn['reaction_center'][0])
#      for k, rxn in list(decarb.items())
# ]

# legends, smiles, rcs = zip(*stuff)
# n_cols = 4
# sub_img_size = (500, 300)
# show_highlighted_grid(smiles, rcs, legends, n_cols, sub_img_size)


In [ ]:
imt_rules = list(set(chain(*[rxn['imt_rules'] for rxn in decarb.values()])))

@interact
def show_imt_groups(imt = widgets.Dropdown(value=imt_rules[0], options=imt_rules)):
    rxns = {k: v for k, v in decarb.items() if v['imt_rules'][0] == imt}
    print(f"Number reactions: {len(rxns)}")
    stuff = [
        (k, rxn['smarts'].split('>>')[0], rxn['reaction_center'][0])
        for k, rxn in list(rxns.items())
    ]

    legends, smiles, rcs = zip(*stuff)
    n_cols = 3
    sub_img_size = (700, 300)
    show_highlighted_grid(smiles, rcs, legends, n_cols, sub_img_size)


interactive(children=(Dropdown(description='imt', options=('rule0024_22', 'rule0024_44', 'rule0024_36', 'rule0…

In [ ]:
# '''
# mol to nx
# get k hop ego graph for all rc atoms
# take union of ego graphs
# convert back to mol / smiles


# Worth converting to networkx at all?,

# What does equal mean? Isomorphic?
# How much flexibility to I want in determining equal?

# '''

# def mol_to_nx(mol):
#     G = nx.Graph()

#     for atom in mol.GetAtoms():
#         G.add_node(atom.GetIdx(),
#                    atomic_num=atom.GetAtomicNum(),
#                    is_aromatic=atom.GetIsAromatic(),
#                    atom_symbol=atom.GetSymbol())
        
#     for bond in mol.GetBonds():
#         G.add_edge(bond.GetBeginAtomIdx(),
#                    bond.GetEndAtomIdx(),
#                    bond_type=bond.GetBondType())
        
#     return G